In [1]:
import altair as alt
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings('ignore')

alt.renderers.enable('notebook')
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

In [ ]:
df = pd.read_csv('arab_observer_1963_1966_hathi.csv')

df['year'] = ''
df.year[df['title'].str.contains('1963')] = '1963'
df.year[df['title'].str.contains('1964')] = '1964'
df.year[df['title'].str.contains('1965')] = '1965'
df.year[df['title'].str.contains('1966')] = '1966'
df['year_date'] = pd.to_datetime(df['year'], format='%Y', errors='ignore')

df.to_csv('year_ao_hathi.csv')
df_grouped_counts = df.groupby(['title', 'year', 'year_date'])['tokenized_counts'].sum().reset_index()
df_grouped_pages = df.groupby(['title', 'year', 'year_date'])['page_number'].count().reset_index()
df_1 = pd.merge(df_grouped_counts, df_grouped_pages, on=['title', 'year', 'year_date'])
df_1.to_csv('grouped_ao_hathi.csv')

melted_counts = pd.melt(df_1, id_vars=['title', 'year', 'year_date'], value_name='value')
melted_counts.to_csv('melted_ao_hathi.csv')

# melted_counts = melted_counts.sort_values(by['year_date'])
# alt.Chart(melted_counts).mark_line(povubuint=True, opacity=0.3).encode(
#     x='title',
#     y='value',
#     color='variable'
# )

# df_1 = df_1.sort_values(by['year_date'])
# alt.Chart(df_1).mark_point().encode(
#     x='page_number',
#     y='tokenized_counts',
#     color='title'
# )

In [ ]:
terms = ['patrice','congo','lumumba','tshombe','leopoldville', 'belgian','mobutu','kasavubu','katanga']
frames = []
for index, row in df.iterrows():
    d = pd.DataFrame(input_df.iloc[index]).transpose()
    d.reset_index(drop=True, inplace=True)
    for t in terms:
        text = df.loc[df.tokenized_text.str.contains(t, regex=False) == True]
        if len(text) > 0:
            counts = df.tokenized_text.apply(lambda x: x.count(t))
            if int(counts) > 0:
                text['term'] = t
                text['word_counts'] = counts
                text.reset_index(drop=True, inplace=True)
                frames.append(text)

ner_df = pd.concat(frames, ignore_index=True)
ner_df.to_csv('congo_ao_hathi.csv')

In [ ]:
# ## Distributions by individual pages
# alt.Chart(new_df).mark_circle(
#     opacity=0.8,
#     stroke='black',
#     strokeWidth=1
# ).encode(
#     alt.X('title', axis=alt.Axis(labelAngle=0)),
#     alt.Y('term:N'),
#     alt.Size('word_counts:Q',
#         scale=alt.Scale(range=[0, 500]),
#         legend=alt.Legend(title='Number of occurences')
#     ),
#     alt.Color('term:N'),
#     tooltip=['term', 'word_counts', 'title']
# ).properties(
#     width=480,
#     height=350
# )

In [ ]:
grouped_df = ner_df.groupby(['title', 'term', 'year', 'year_date'])['word_counts'].sum().reset_index()
# grouped_df['string_date'] = grouped_df.date.astype(str)
grouped_pages = ner_df.groupby(['title', 'term', 'year', 'year_date'])['page_number'].count().reset_index()
# grouped_pages['string_date'] = grouped_pages.date.astype(str)
co = pd.merge(grouped_df, grouped_pages, on=['title', 'term', 'year', 'year_date'])

In [ ]:
### Grouped by issues essentially 
# alt.Chart(co).mark_circle(
#     opacity=0.8,
#     stroke='black',
#     strokeWidth=1
# ).encode(
#     alt.X('title', axis=alt.Axis(labelAngle=0)),
#     alt.Y('term:N'),
#     alt.Size('word_counts:Q',
#         scale=alt.Scale(range=[0, 500]),
#         legend=alt.Legend(title='Number of occurences')
#     ),
#     alt.Color('term:N'),
#     tooltip=['term', 'word_counts', 'title']
# ).properties(
#     width=480,
#     height=350
# )

In [ ]:
# alt.Chart(new_df).mark_bar().encode(
#     x='title',
#     y='sum(word_counts)',
#     color='term'
# )

In [ ]:
# highlight = alt.selection(type='single', on='mouseover',
#                           fields=['term'], nearest=True)

# base = alt.Chart(co).encode(
#     x='title',
#     y='word_counts',
#     color='term:N',
#     size='page_number'
# )

# points = base.mark_circle().encode(
#     opacity=alt.value(0.6)
# ).add_selection(
#     highlight
# ).properties(
#     width=600
# )

# lines = base.mark_line().encode(
#     size=alt.condition(~highlight, alt.value(1), alt.value(3))
# )

# points + lines

In [ ]:
# alt.Chart(co).mark_line().encode(
#     x='title',
#     y='word_counts:Q',
#     color='term:N',
#     row=alt.Row('term:N'
#     )
# ).properties(height=50, width=400)

In [ ]:
# alt.Chart(co).mark_circle().encode(
#     x='word_counts',
#     y='page_number',
#     color='term',
# )

In [ ]:
# counts = co
# pts = alt.selection(type="single", fields=['title'])
# color = alt.condition(pts,
#                       alt.Color('term:N'),
#                       alt.value('lightgray'))

# lines = alt.Chart(counts).mark_line(point=True, opacity=0.3).encode(
#     x=alt.X('title'),
#     y='word_counts:Q',
#     color=color,
#     tooltip=['title', 'term', 'word_counts']
# ).properties(
#     title='Issue similarity across all words',
#     width=700,
#     height=300,
# )
# x_ticks = alt.Chart(counts).mark_bar(opacity=0.7).encode(
#     x=alt.X('title'),
#     y=alt.Y('page_number'),
#     color=color,
#     tooltip=['title', 'word_counts']
# ).properties(
#     width=700,
#     height=100,
#     selection=pts,
#     title='# words'
# )
# (lines & x_ticks)